In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import os
import cv2
import numpy as np
import json


# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

# Load the trained model
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss=tf.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

# Load the trained weights
model.load_weights('models/notesmodel.h5')

# Set the directories for the images and for the output
image_dir = 'test_images'
output_dir_nonnotes = 'output/non_notes'
output_dir_notes = 'output/notes'
output_dir_results = 'output/results'


# Create the directory if it does not exist

os.makedirs(output_dir_nonnotes, exist_ok=True)
os.makedirs(output_dir_notes, exist_ok=True)
os.makedirs(output_dir_results, exist_ok=True)


# Create dictionaries to store the evaluation results
results_dict = {}
results_dict['notes'] = []
results_dict['non-notes'] = []



# Iterate through the images in the input directory
for image_name in os.listdir(image_dir):
  # Load the image and preprocess it
  image_path = os.path.join(image_dir, image_name)
  image = cv2.imread(image_path)
  image = cv2.resize(image, (256, 256))
  image = image / 255.0

  # Predict the class of the image
  prediction = model.predict(np.array([image]))[0,0]
  prediction = model.predict(np.array([image]))


  # Evaluate the prediction and save the results
  if prediction > 0.5:
    result = {'image': image_name, 'class': 'non-notes', 'score': float(prediction)}
    results_dict['non-notes'].append(result)
    os.rename(image_path, os.path.join(output_dir_nonnotes, image_name))
  else:
    result = {'image': image_name, 'class': 'notes', 'score': float(1-prediction)}
    results_dict['notes'].append(result)
    os.rename(image_path, os.path.join(output_dir_notes, image_name))


# Save the evaluation results for the image
  result_path = os.path.join(output_dir_results, os.path.splitext(image_name)[0])
 
  cv2.imwrite(result_path+'.png', image*255)
  

# Save the overall evaluation results
results_path = os.path.join(output_dir_results, 'results.json')
with open(results_path, 'w') as f:
  json.dump(results_dict, f)


1/1 [==============================] - 0s 22ms/step
